In [ ]:
#1. Graph-Based Collaborative Filtering Model

# Graph-Based Collaborative Filtering Model

# Import necessary libraries
import pandas as pd
import networkx as nx
from node2vec import Node2Vec
import numpy as np

# Load and preprocess the Movielens dataset
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Create a bipartite graph from the user-movie interactions
B = nx.Graph()
B.add_nodes_from(ratings['user_id'], bipartite=0)  # Add user nodes
B.add_nodes_from(ratings['movie_id'], bipartite=1)  # Add movie nodes
edges = list(ratings[['user_id', 'movie_id']].itertuples(index=False, name=None))
B.add_edges_from(edges)

# Apply Node2Vec to learn embeddings
node2vec = Node2Vec(B, dimensions=64, walk_length=30, num_walks=200, workers=4)
model = node2vec.fit(window=10, min_count=1, batch_words=4)

# Function to get recommendations
def get_recommendations(user_id, top_n=10):
    user_embedding = model.wv[str(user_id)]
    similarities = []
    for node in model.wv.vocab:
        if node.startswith('m_'):  # Ensure it's a movie node
            sim = np.dot(user_embedding, model.wv[node])
            similarities.append((node, sim))
    similarities.sort(key=lambda x: x[1], reverse=True)
    return [int(node[2:]) for node, sim in similarities[:top_n]]

# Example usage
print(get_recommendations(1))

#2. Autoencoder-Based Collaborative Filtering Model

# Autoencoder-Based Collaborative Filtering Model

# Import necessary libraries
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder

# Load and preprocess the Movielens dataset
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
n_users = ratings['user_id'].nunique()
n_movies = ratings['movie_id'].nunique()

# Create a user-item interaction matrix
interaction_matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

# Normalize the data
interaction_matrix_norm = (interaction_matrix - interaction_matrix.min()) / (interaction_matrix.max() - interaction_matrix.min())
interaction_matrix_norm = interaction_matrix_norm.values

# Define the autoencoder model
input_dim = interaction_matrix_norm.shape[1]
encoding_dim = 64

input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
autoencoder.fit(interaction_matrix_norm, interaction_matrix_norm, epochs=50, batch_size=256, shuffle=True, validation_split=0.2)

# Get the user embeddings
encoder = Model(input_layer, encoded)
user_embeddings = encoder.predict(interaction_matrix_norm)

# Function to get recommendations
def get_recommendations(user_id, top_n=10):
    user_idx = ratings[ratings['user_id'] == user_id].index[0]
    user_embedding = user_embeddings[user_idx]
    scores = np.dot(user_embedding, user_embeddings.T)
    recommendations = np.argsort(scores)[-top_n:]
    return recommendations

# Example usage
print(get_recommendations(1))

#3. Reinforcement Learning-Based Recommendation Model

# Reinforcement Learning-Based Recommendation Model

# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load and preprocess the Movielens dataset
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Define state and action space
n_users = ratings['user_id'].nunique()
n_movies = ratings['movie_id'].nunique()
state_space = n_users
action_space = n_movies

# Define Q-learning model
model = Sequential()
model.add(Dense(64, input_dim=state_space, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(action_space, activation='linear'))
model.compile(loss='mse', optimizer='adam')

# Q-learning parameters
gamma = 0.95
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995

# Train Q-learning model
for epoch in range(1000):
    for index, row in ratings.iterrows():
        state = np.zeros(state_space)
        state[row['user_id'] - 1] = 1
        action = np.zeros(action_space)
        action[row['movie_id'] - 1] = 1
        reward = row['rating']
        next_state = np.zeros(state_space)
        if np.random.rand() <= epsilon:
            action_index = np.random.choice(action_space)
        else:
            action_index = np.argmax(model.predict(state)[0])
        next_state[row['user_id'] - 1] = 1
        target = reward + gamma * np.max(model.predict(next_state)[0])
        target_f = model.predict(state)
        target_f[0][action_index] = target
        model.fit(state, target_f, epochs=1, verbose=0)
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

# Function to get recommendations
def get_recommendations(user_id, top_n=10):
    state = np.zeros(state_space)
    state[user_id - 1] = 1
    action_scores = model.predict(state)[0]
    recommendations = np.argsort(action_scores)[-top_n:]
    return recommendations

# Example usage
print(get_recommendations(1))
